In [ ]:
# Install pykeen - package that is used to set up the pipeline
pip install pykeen

In [ ]:
# Mount Google Drive to work on relevant files
from google.colab import drive
drive.mount('/content/drive')

# Import necessary packages
import torch
from typing import Optional, Dict
from pykeen.models import RGCN
from pykeen.nn.modules import Interaction
from pykeen.triples import TriplesFactory
import numpy as np

class WeightedRGCN(RGCN):
    def __init__(
        self,
        triples_factory: TriplesFactory,
        embedding_dim: int = 64,
        num_layers: int = 2,
        edge_attributes: Optional[Dict[tuple, Dict[str, float]]] = None,
        dropout: float = 0.3,
        **kwargs
    ):
        super().__init__(
            triples_factory=triples_factory,
            embedding_dim=embedding_dim,
            num_layers=num_layers,
            **kwargs
        )

        # store edge attributes or initialize empty if None
        self.edge_attributes = edge_attributes or {}

        # define the attributes that will be processed for each edge
        self.string_attributes = [
            'coexpression',
            'experimentally_determined_interaction',
            'automated_textmining',
            'combined_score',
            'rmsd_score'
        ]

        # create NNs for processing each attribute type
        # each network converts a single attribute value into a weight
        self.attr_processors = torch.nn.ModuleDict({
            attr: torch.nn.Sequential(
                torch.nn.Linear(1, 8),
                torch.nn.ReLU(),
                torch.nn.Dropout(dropout),
                torch.nn.Linear(8, 1),
                torch.nn.Sigmoid()
            ) for attr in self.string_attributes
        })

        # attention combines all processed attributes into a single weight
        self.attr_attention = torch.nn.Sequential(
            torch.nn.Linear(len(self.string_attributes), 1),
            torch.nn.Sigmoid()
        )

        self.gradient_checkpointing = True

    def _get_edge_weights(self, h_idx, r_idx, t_idx):
        """
        Compute edge weights in a memory-efficient manner by processing data in chunks.
        """
        batch_size = len(h_idx)
        device = self.device

        # process data in chunks to reduce memory usage
        chunk_size = 1000
        all_weights = []

        # process each chunk
        for i in range(0, batch_size, chunk_size):
            chunk_h = h_idx[i:i + chunk_size]
            chunk_r = r_idx[i:i + chunk_size]
            chunk_t = t_idx[i:i + chunk_size]

            # initialize attribute values with default of 0.5
            chunk_attr_values = torch.ones(len(chunk_h), len(self.string_attributes), device=device) * 0.5

            # process each triple in the chunk
            for j, (h, r, t) in enumerate(zip(chunk_h, chunk_r, chunk_t)):
                # Get attributes for current edge
                edge_attrs = self.edge_attributes.get((h.item(), r.item(), t.item()), {})

                # process each attribute
                for k, attr in enumerate(self.string_attributes):
                    value = edge_attrs.get(attr, 0.5)  # Default to 0.5 if attribute missing
                    if isinstance(value, str) and value == 'N/A':
                        value = 0.5  # Handle N/A values
                    chunk_attr_values[j, k] = float(value)

            # process each attribute through its NN
            processed_attrs = []
            for attr_idx, attr in enumerate(self.string_attributes):
                attr_val = chunk_attr_values[:, attr_idx:attr_idx+1]
                processed_val = self.attr_processors[attr](attr_val)
                processed_attrs.append(processed_val)

            # combine processed attributes using attention mechanism
            chunk_combined = torch.cat(processed_attrs, dim=1)
            chunk_weights = self.attr_attention(chunk_combined)
            all_weights.append(chunk_weights)

        # combine all chunk weights and remove extra dimensions
        return torch.cat(all_weights, dim=0).squeeze(-1)

    def forward(
        self,
        h_indices: torch.LongTensor,
        r_indices: torch.LongTensor,
        t_indices: torch.LongTensor,
        mode: Optional[str] = None,
    ) -> torch.FloatTensor:
        """
        Forward pass of the model, combining RGCN scores with computed edge weights.
        """
        if mode is None:

            # training mode: compute full weights
            if self.gradient_checkpointing and self.training:

                # use gradient checkpointing to save memory during training
                weights = torch.utils.checkpoint.checkpoint(
                    self._get_edge_weights,
                    h_indices, r_indices, t_indices
                )
            else:
                weights = self._get_edge_weights(h_indices, r_indices, t_indices)

            # get base RGCN scores and multiply by computed weights
            scores = super().forward(h_indices, r_indices, t_indices)
            return scores * weights
        else:
            # prediction mode: use default weight of 0.5
            scores = super().forward(h_indices, r_indices, t_indices, mode=mode)

In [ ]:
def graph_to_edge_attributes(graph) -> Dict[tuple, Dict[str, float]]:
    """convert NetworkX graph edge attributes to dictionary format."""
    edge_attrs = {}
    for u, v, data in graph.edges(data=True):

        # get relation type
        relation = data.get('relation', 'related_to')

        # extract STRING attributes
        attrs = {
            'coexpression': data.get('coexpression', 'N/A'),
            'experimentally_determined_interaction': data.get('experimentally_determined_interaction', 'N/A'),
            'automated_textmining': data.get('automated_textmining', 'N/A'),
            'combined_score': data.get('combined_score', 'N/A'),
            'rmsd_score': data.get('rmsd_score', 'N/A')
        }

        # store attributes keyed by (head, relation, tail)
        edge_attrs[(u, relation, v)] = attrs

    return edge_attrs

In [ ]:
# Import necessary packages
import pickle
from pykeen.pipeline import pipeline
import torch
import logging
from sklearn.model_selection import train_test_split

def prepare_data(graph_path: str):
    """
    Prepare graph data for training by creating train/test/validation splits of triplets
    """
    # Load the graph
    with open(graph_path, "rb") as f:
        graph = pickle.load(f)

    # Extract triples and edge attributes
    triples = []
    for source, target, data in graph.edges(data=True):
        relation = data.get('relation', 'related_to')
        triples.append((str(source), str(relation), str(target)))

    # Convert to numpy array for splitting
    triples_array = np.array(triples)

    # Create train/test split first
    train_val_triples, test_triples = train_test_split(
        triples_array,
        test_size=0.1,
        random_state=42
    )

    # Create train/validation split
    train_triples, val_triples = train_test_split(
        train_val_triples,
        test_size=0.11111,
        random_state=42
    )

    # Create factories
    training_factory = TriplesFactory.from_labeled_triples(train_triples)
    validation_factory = TriplesFactory.from_labeled_triples(val_triples)
    testing_factory = TriplesFactory.from_labeled_triples(test_triples)

    # Get edge attributes
    edge_attributes = graph_to_edge_attributes(graph)

    return training_factory, testing_factory, validation_factory, edge_attributes

In [ ]:
# Path to graph
graph_path = "/content/drive/MyDrive/Colabs/data/graph_w_covid_genes_treatments_string_cif_coalesced_3.pkl"

# Prepare data
train_factory, test_factory, val_factory, edge_attrs = prepare_data(graph_path)

In [ ]:
# Import necessary packages for pipeline
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from typing import Dict, List, Tuple
import torch
from pykeen.predict import predict_target

def train_weighted_rgcn(
    training_factory: TriplesFactory,
    testing_factory: TriplesFactory,
    validation_factory: TriplesFactory,
    edge_attributes: Dict,
) -> Dict:
    """
    Create a RGCN pipeline to train the model
    """
    return pipeline(
        model=MemoryEfficientWeightedRGCN,
        model_kwargs={
            'edge_attributes': edge_attributes,
            'embedding_dim': 100,  # Dimension of embeddings
            'num_layers': 2,       # Number of RGCN layers
        },
      training_kwargs={
          'num_epochs': 50,
          'batch_size': 6000,
      },
        training=training_factory,
        testing=testing_factory,
        validation=validation_factory,
        training_loop='slcwa',
        optimizer='adam',
        device='cuda'
    )

# Train on CPU with sampled data
result = train_weighted_rgcn(
    training_factory=train_factory,
    testing_factory=test_factory,
    validation_factory=val_factory,
    edge_attributes=edge_attrs
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted 5050817 triples from the graph.


INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.message_passing:No num_bases was provided. Using sqrt(num_relations)=10.
INFO:pykeen.nn.message_passing:No num_bases was provided. Using sqrt(num_relations)=10.
INFO:pykeen.nn.message_passing:No num_bases was provided. Using sqrt(num_relations)=10.
INFO:pykeen.nn.message_passing:No num_bases was provided. Using sqrt(num_relations)=10.
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(10, 10000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(10, 10000)
      )
    )
  )
  (self_loop): Linear(in_features=100, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embeddi

Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/674 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/674 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/674 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/674 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/674 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/505k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 345.75s seconds


In [ ]:
# Import necessary modules
import os
import pickle

# Retrieve the trained model from the pipeline result
model = result

# Define the path where the model will be saved
save_path = '/content/drive/MyDrive/my_models/trained_rgcn_new_graph_model_pipeline_result_weights.pkl'
directory = os.path.dirname(save_path)

# Check if the directory exists, and if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the model to the specified path using pickle
with open(save_path, 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Get evaluation results
results = result.metric_results.to_dict()
print("\nModel Evaluation Results:")
print(results)

# Get embeddings correctly
entity_embeddings = pipeline_result.model.entity_representations[0](indices=None).cpu().detach().numpy()
relation_embeddings = pipeline_result.model.relation_representations[0](indices=None).cpu().detach().numpy()


Model Evaluation Results:
{'head': {'optimistic': {'adjusted_geometric_mean_rank_index': -1.205822912937747, 'z_arithmetic_mean_rank': -851.0719297755124, 'median_absolute_deviation': 7797.746368230213, 'z_geometric_mean_rank': -858.9264914320823, 'inverse_geometric_mean_rank': 1.2793998710895143e-05, 'adjusted_inverse_harmonic_mean_rank': -0.00010848016987911259, 'variance': 286763808.1124392, 'inverse_median_rank': 1.133780420745914e-05, 'inverse_arithmetic_mean_rank': 1.227691131859031e-05, 'adjusted_arithmetic_mean_rank_index': -0.6913997360030824, 'geometric_mean_rank': 78161.64614339203, 'harmonic_mean_rank': 59985.28350980916, 'z_inverse_harmonic_mean_rank': -18.66165517941274, 'inverse_harmonic_mean_rank': 1.667075558351698e-05, 'adjusted_arithmetic_mean_rank': 1.6913853790956501, 'count': 505082.0, 'standard_deviation': 16934.101928134223, 'arithmetic_mean_rank': 81453.71209823355, 'median_rank': 88200.5, 'hits_at_1': 0.0, 'hits_at_3': 0.0, 'hits_at_5': 0.0, 'hits_at_10': 3.9

In [ ]:
# Define the list of COVID-related diseases
COV_disease_list = [
    "Disease::SARS-CoV2 E",
    "Disease::SARS-CoV2 M",
    "Disease::SARS-CoV2 N",
    "Disease::SARS-CoV2 Spike",
    "Disease::SARS-CoV2 nsp1",
    "Disease::SARS-CoV2 nsp10",
    "Disease::SARS-CoV2 nsp11",
    "Disease::SARS-CoV2 nsp12",
    "Disease::SARS-CoV2 nsp13",
    "Disease::SARS-CoV2 nsp14",
    "Disease::SARS-CoV2 nsp15",
    "Disease::SARS-CoV2 nsp2",
    "Disease::SARS-CoV2 nsp4",
    "Disease::SARS-CoV2 nsp5",
    "Disease::SARS-CoV2 nsp5_C145A",
    "Disease::SARS-CoV2 nsp6",
    "Disease::SARS-CoV2 nsp7",
    "Disease::SARS-CoV2 nsp8",
    "Disease::SARS-CoV2 nsp9",
    "Disease::SARS-CoV2 orf10",
    "Disease::SARS-CoV2 orf3a",
    "Disease::SARS-CoV2 orf3b",
    "Disease::SARS-CoV2 orf6",
    "Disease::SARS-CoV2 orf7a",
    "Disease::SARS-CoV2 orf8",
    "Disease::SARS-CoV2 orf9b",
    "Disease::SARS-CoV2 orf9c",
    "Disease::MESH:D045169",
    "Disease::MESH:D045473",
    "Disease::MESH:D001351",
    "Disease::MESH:D065207",
    "Disease::MESH:D028941",
    "Disease::MESH:D058957",
    "Disease::MESH:D006517",
]


In [ ]:
def get_top_predictions(
    model,
    training_factory,
    relation: str,
    tail_entities: List[str],
    k: int = 20
) -> dict:
    """
    Get top 20 compund predictions for given tail (COVID) entities and relation.
    """
    results = {}

    for tail in tail_entities:
        try:
            # Use predict_target to get predictions
            predictions = predict_target(
                model=model,
                tail=tail,
                relation=relation,
                triples_factory=training_factory
            )

            # Get the results as a dataframe and apply the filter
            df = predictions.df

            # Filter for only Compound predictions
            compound_df = df[df['head_label'].str.startswith('Compound::')].sort_values('score', ascending=False)

            if compound_df.empty:
                print(f"Warning: No compound predictions found for {tail}")
                continue

            # Get top k predictions
            top_k_df = compound_df.head(k)

            # Convert to list of (compound, score) tuples
            head_scores = list(zip(top_k_df['head_label'], top_k_df['score']))
            results[tail] = head_scores

        except KeyError as e:
            print(f"Warning: Entity or relation not found in training data: {e}")
            continue

        except Exception as e:
            print(f"Error predicting for {tail}: {str(e)}")
            continue

    return results

# Get predictions for COVID diseases
relation = "DRKG::Treats::Compound:Disease"
predictions = get_top_predictions(
    pipeline_result.model,
    training_factory,
    relation,
    COV_disease_list
)

# Print results
print("\nTop 20 predicted compounds for each disease:")
for disease, compounds in predictions.items():
    if compounds:  # Only print if we have compound predictions
        print(f"\n{disease}:")
        for i, (compound, score) in enumerate(compounds, 1):
            print(f"{i}. {compound} (score: {score:.4f})")


Top 20 predicted compounds for each disease:

Disease::SARS-CoV2 E:
1. Compound::CHEMBL432600 (score: 0.6069)
2. Compound::CHEMBL2109193 (score: 0.5830)
3. Compound::DB07384 (score: 0.5689)
4. Compound::brenda:104600 (score: 0.5578)
5. Compound::pubchem:407294 (score: 0.5544)
6. Compound::MESH:C403563 (score: 0.5312)
7. Compound::DB05033 (score: 0.5189)
8. Compound::brenda:226203 (score: 0.5130)
9. Compound::DB06098 (score: 0.4925)
10. Compound::DB08629 (score: 0.4732)
11. Compound::zinc:ZINC000005650745 (score: 0.4598)
12. Compound::DB06734 (score: 0.4454)
13. Compound::CHEMBL2109474 (score: 0.4336)
14. Compound::DB01790 (score: 0.4263)
15. Compound::DB03478 (score: 0.4078)
16. Compound::DB06103 (score: 0.3818)
17. Compound::DB07546 (score: 0.3799)
18. Compound::brenda:222144 (score: 0.3651)
19. Compound::CHEMBL460785 (score: 0.3649)
20. Compound::DB12109 (score: 0.3613)

Disease::SARS-CoV2 M:
1. Compound::DB06098 (score: 1.3097)
2. Compound::CHEMBL3137353 (score: 0.8452)
3. Compound

In [ ]:
def get_compounds_with_treatments(graph):
    """
    Get list of compounds that have treatment data associated with COVID-19.
    Returns list of compounds and count.
    """
    # Initialize an empty list to store compound nodes.
    compounds = []

    # Iterate through all nodes and their attributes.
    for node, attributes in graph.nodes(data=True):
        # Check if the node is a string, starts with 'Compound::', and contains 'treatment_data' in its attributes.
        if (isinstance(node, str) and
            node.startswith('Compound::') and
            'treatment_data' in attributes):
            compounds.append(node)

    return compounds, len(compounds)

# Get the compounds and count
compounds, num_compounds = get_compounds_with_treatments(graph)
print(compounds)
print(num_compounds)

['Compound::DB01087', 'Compound::DB01072', 'Compound::DB06817', 'Compound::DB00773', 'Compound::DB01392', 'Compound::DB00539', 'Compound::DB07795', 'Compound::DB13997', 'Compound::DB09299', 'Compound::DB00608', 'Compound::DB14126', 'Compound::DB00091', 'Compound::DB00198', 'Compound::DB00811', 'Compound::DB09026', 'Compound::DB08934', 'Compound::DB00220', 'Compound::DB11779', 'Compound::DB00328', 'Compound::DB04786', 'Compound::DB00482', 'Compound::DB01015', 'Compound::DB09102', 'Compound::DB01264', 'Compound::DB01645', 'Compound::DB00822', 'Compound::DB09065', 'Compound::DB12610', 'Compound::DB00831', 'Compound::DB00457', 'Compound::DB04115', 'Compound::DB01041', 'Compound::DB00722', 'Compound::DB00213', 'Compound::DB00133', 'Compound::DB06803', 'Compound::DB12270', 'Compound::DB00319', 'Compound::DB11952', 'Compound::DB00302', 'Compound::DB00199', 'Compound::DB13925', 'Compound::DB00143', 'Compound::DB09061', 'Compound::DB14810', 'Compound::DB00673', 'Compound::DB00159', 'Compound::D

In [ ]:
def analyze_predictions_vs_treatments(predictions, actual_compounds):
    """
    Analyze how many predicted compounds for each disease are actual treatment compounds.
    """
    analysis_results = {}

    # Loop through all predicted compounds and respective diseases
    for disease, predicted_compounds in predictions.items():
        if predicted_compounds:
            # Get just the compound names from predictions (without scores)
            predicted_set = set(compound for compound, _ in predicted_compounds)

            # Find overlap with actual compounds
            overlap = predicted_set.intersection(set(actual_compounds))

            # Calculate percentage
            percentage = (len(overlap) / len(predicted_compounds)) * 100

            # Store results
            analysis_results[disease] = {
                'overlap_count': len(overlap),
                'overlap_percentage': percentage,
                'overlapping_compounds': list(overlap)
            }

            # Print results
            print(f"\n{disease}:")
            print(f"Number of predicted compounds in treatment database: {len(overlap)}/20")
            print(f"Percentage: {percentage:.2f}%")
            print("Overlapping compounds:")
            for compound in overlap:
                print(f"- {compound}")

    return analysis_results

# First get predictions
relation = "DRKG::Treats::Compound:Disease"
predictions = get_top_predictions(
    pipeline_result.model,
    training_factory,
    relation,
    COV_disease_list
)

# Then get actual treatment compounds
actual_compounds, num_actual = get_compounds_with_treatments(graph)

# Analyze overlap
print(f"\nTotal number of compounds with actual treatment data: {num_actual}")
analysis = analyze_predictions_vs_treatments(predictions, actual_compounds)


Total number of compounds with actual treatment data: 451

Disease::SARS-CoV2 E:
Number of predicted compounds in treatment database: 0/20
Percentage: 0.00%
Overlapping compounds:

Disease::SARS-CoV2 M:
Number of predicted compounds in treatment database: 0/20
Percentage: 0.00%
Overlapping compounds:

Disease::SARS-CoV2 N:
Number of predicted compounds in treatment database: 1/20
Percentage: 5.00%
Overlapping compounds:
- Compound::DB00877

Disease::SARS-CoV2 Spike:
Number of predicted compounds in treatment database: 8/20
Percentage: 40.00%
Overlapping compounds:
- Compound::DB00091
- Compound::DB01098
- Compound::DB14513
- Compound::DB00602
- Compound::DB00468
- Compound::DB00877
- Compound::DB00916
- Compound::DB00563

Disease::SARS-CoV2 nsp1:
Number of predicted compounds in treatment database: 0/20
Percentage: 0.00%
Overlapping compounds:

Disease::SARS-CoV2 nsp10:
Number of predicted compounds in treatment database: 8/20
Percentage: 40.00%
Overlapping compounds:
- Compound::DB00